<a href="https://colab.research.google.com/github/Skvorok/netology_pyda/blob/master/Hw13_pd_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Задание 1**

Используйте файл с оценками фильмов ml-latest-small/ratings.csv. Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок. Под временем жизни понимается разница между максимальным и минимальным значением столбца timestamp для данного значения userId.


In [0]:
import platform
if platform.system() == 'Linux':
  from google.colab import drive
  drive.mount('/content/drive')
  trajectory = '/content/drive/My Drive/Colab Notebooks/ml-latest-small/'
else:
  trajectory = ''

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import pandas as pd
import datetime

In [0]:
rating = pd.read_csv(trajectory+'ratings.csv')
x = 100    #количество оценок
rating['date'] = rating.timestamp.apply(lambda x: pd.Timestamp(x, unit = 's'))
middle = rating.groupby('userId')
rating_count = middle.count().join(middle.min().date, rsuffix = '_min').join(middle.max().date, rsuffix = '_max')
rating_count['difference'] = rating_count.date_max-rating_count.date_min
rating_count[middle.rating.count() > x].difference.mean()
print(f'Среднее время жизни пользователей, которые выставили более 100 оценок - {rating_count[middle.rating.count() > x].difference.mean()}')


Среднее время жизни пользователей, которые выставили более 100 оценок - 517 days 17:22:47.734693




**Задание 2**

Дана статистика услуг перевозок клиентов компании по типам (см. файл с кодом занятия). Необходимо сформировать две таблицы:
таблицу с тремя типами выручки для каждого client_id без указания адреса клиента
аналогичную таблицу по типам выручки с указанием адреса клиента
Обратите внимание, что в процессе объединения таблиц данные не должны теряться.



In [0]:
# Дана статистика услуг перевозок клиентов компании по типам:
# 1)rzd - железнодорожные перевозки
# 2)auto - автомобильные перевозки
# 3)air - воздушные перевозки
# 4)client_base - адреса клиентов

In [0]:
rzd = pd.DataFrame({'client_id': [111, 112, 113, 114, 115],
                    'rzd_revenue': [1093, 2810, 10283, 5774, 981]})

auto = pd.DataFrame({'client_id': [113, 114, 115, 116, 117],
                     'auto_revenue': [57483, 83, 912, 4834, 98]})

air = pd.DataFrame({'client_id': [115, 116, 117, 118],
                    'air_revenue': [81, 4, 13, 173]})

client_base = pd.DataFrame({'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
                            'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                                        'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']})

first_table = rzd.merge(auto, how = 'outer').merge(air, how = 'outer')
second_table = first_table.merge(client_base, how = 'outer')

print(first_table, '\n\n\n', second_table)


   client_id  rzd_revenue  auto_revenue  air_revenue
0        111       1093.0           NaN          NaN
1        112       2810.0           NaN          NaN
2        113      10283.0       57483.0          NaN
3        114       5774.0          83.0          NaN
4        115        981.0         912.0         81.0
5        116          NaN        4834.0          4.0
6        117          NaN          98.0         13.0
7        118          NaN           NaN        173.0 


    client_id  rzd_revenue  auto_revenue  air_revenue          address
0        111       1093.0           NaN          NaN  Комсомольская 4
1        112       2810.0           NaN          NaN   Энтузиастов 8а
2        113      10283.0       57483.0          NaN  Левобережная 1а
3        114       5774.0          83.0          NaN          Мира 14
4        115        981.0         912.0         81.0        ЗЖБИиДК 1
5        116          NaN        4834.0          4.0    Строителей 18
6        117          NaN    

**Задание 3**

В задаче сквозной аналитики вам предоставили данные по местоположению пользователей. Т. е. для каждого user_id известна последовательность координат (широта/долгота), когда они требовались приложению для полноценной работы. Как бы вы добавили эти сведения в таблицу визитов и покупок? Для составления ответа можно использовать вопросы:
У каждого пользователя известен набор координат. А для связывания с визитом или фактом покупки скорее всего потребуется одно-два числа. Как их получить?
Наборы координат одного и того же пользователя могут быть значительно удалены друг от друга. Как это отразится на вопросе расчетах пункта 1?
Какие дополнительные признаки можно получить из координат? Ведь это просто числа, которые сами по себе мало что дают.



*   Вводные данные: user_id и его координаты (широта/долгота).
*   Для добавления координатных сведений user_id в таблицу "Визиты и покупки" возможно было бы применить функцию merge (склеивание таблиц: таблица 1 - это данные о координатах юзеров, таблица 2 - это числовые значения визитов и покупок юзеров). 
*   Т.к.набор координат представляет собой пару чисел, стоит подумать как эту пару конвертировать в одно число. Для решения этого вопроса, можно представить координаты, как географическую конкретную локацию в виде числа, символизирующего город, регион, область, населенный пункт и т.д.
*   
Если возникает ситуация, когда набор координат одного и того же пользователя удалены друг от друга, то в сведениях о визитах и покупках будут указываться разные данные. Например, разная геолокация одного пользователя может влиять на параметр визита (или его отсутствия), но не повлиять на покупку (она может совершится). 
*   Из координат можно получить дополнительные сведения, если есть на то поставленная задача. Например, зная точное место георасположения пользователя, можно выяснить информацию о его вероятных ресурсных возможностях: живя в мегаполисе, ресурсов (финансовых) больше для совершения той или иной покупки, нежели живя в провинции. Здесь важно понимать целевое направление приложения и исходить из точных данных. 

